This notebook runs the original, one-dimensional AxiSEM code (https://github.com/geodynamics/axisem) from start to finish.

Everything it does could also be done simply on the command line, but I found this a useful way to automate/centralise input parameters and make the process less fiddly early in my PhD.

You should only need to edit the parameters in the second cell, provided you follow the AxiSEM file structure and have pointed the notebook to the mesher and solver directories.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from subprocess import run
import re

Manually set variables and path names - edit this cell!

In [55]:
### inparam_mesh ###
#background model e.g. prem_iso, ak135, iasp91, external
bg_model = 'external'
ext_model = "github_ice_shell.bm"
dom_period ='0.5'
#FSF = 0.9
radius = '2.52e+5'#m
c_layers = '0'

### inparam_basic ###
#sim_type e.g. single, moment, force
sim_type = 'single'
duration = '200' #s
#rec_type e.g. colatlon or stations
rec_type = 'colatlon'
run_name = 'github_2Hz_test13'

lat_hetero = 'False'
attenuation = 'False'
save_snapshots = 'False'

### inparam_source ###
#source type e.g. mrr, explosion, mtt_p_mpp, vertforce | mtr, mpr, thetaforce, phiforce | mtp, mtt_m_mpp
source_type = 'mrr'
source_dep = '5.0'
ev_lat = '90.0'
ev_lon = '0.0'
source_amp = '1E13'

### inparam_advanced ###
#source function e.g. errorf, gauss_0, gauss_1, gauss_2, quheavi, dirac_0
source_function = 'gauss_0'
# only relevant if save_snapshots = true
snapshot_dt = '5.'
snapshots_format = 'vtk'
# ATTENUATION
f_min = '0.01'
f_max = '10'
f_ref = '1.0'

### CMTSOLUTION ###
#takes source_dep, ev_lat, and ev_lon in the case that sim_type = 'moment'
Mrr = '4.710000e+24'
Mtt = '3.810000e+22'
Mpp = '-4.740000e+24'
Mrt = '3.990000e+23'
Mrp = '-8.050000e+23'
Mtp = '-1.230000e+24'

### let python know where the MESHER and SOLVER directories are ###
MESHER = "./axisem-code/MESHER"
SOLVER = "./axisem-code/SOLVER"

Implement all those variables

In [56]:
with open(f"{MESHER}/inparam_mesh", 'r') as fin:
    file = fin.read()
    #lines represents the full text of the original file
    lines = file.splitlines()
    #check every line in turn for the text:
    for i in range(0, len(lines)):
        if 'BACKGROUND_MODEL' in lines[i]:
            p = lines[i].split()
            #replace the second entry on the line with the new background model
            lines[i] = lines[i].replace(p[1], bg_model)
        if 'EXT_MODEL' in lines[i] and 'external:' not in lines[i]:
            if ext_model is not None:
                lines[i] = f"EXT_MODEL           {ext_model}"
                print(lines[i])
            if ext_model is None:
                lines[i] = f"# EXT_MODEL           {ext_model}"
                print(lines[i])
        if 'DOMINANT_PERIOD' in lines[i]:
            p = lines[i].split()
            lines[i] = lines[i].replace(p[1], dom_period)
        if 'COARSENING_LAYERS' in lines[i]:
            p = lines[i].split()
            lines[i] = lines[i].replace(p[1], c_layers)
        #if 'FLUID_SHRINKING_FACTOR' in lines[i]:
            #p = lines[i].split()
            #lines[i] = lines[i].replace(p[1], FSF) 
        if 'RADIUS' in lines[i]:
            lines[i] = f"RADIUS             {radius}"

#opens writable file to output updated text; individually writes each line in turn
with open(f"{MESHER}/inparam_mesh", 'w') as fout:
    for item in lines:
        fout.write("%s\n" % item)


EXT_MODEL           github_ice_shell.bm


In [57]:
with open(f"{SOLVER}/inparam_basic", 'r') as fin:
    file2 = fin.read()
    lines2 = file2.splitlines()
    for i in range(0, len(lines2)):
        if 'SIMULATION_TYPE' in lines2[i]:
            p = lines2[i].split()
            lines2[i] = lines2[i].replace(p[1], sim_type)
        if 'SEISMOGRAM_LENGTH' in lines2[i]:
            p = lines2[i].split()
            lines2[i] = lines2[i].replace(p[1], duration)
        if 'RECFILE_TYPE' in lines2[i]:
            p = lines2[i].split()
            lines2[i] = lines2[i].replace(p[1], rec_type) 
        if 'MESHNAME' in lines2[i]:
            p = lines2[i].split()
            lines2[i] = lines2[i].replace(p[1],run_name)
        if 'LAT_HETEROGENEITY' in lines2[i]:
            p = lines2[i].split()
            lines2[i] = lines2[i].replace(p[1], lat_hetero)
        if 'ATTENUATION' in lines2[i]:
            p = lines2[i].split()
            lines2[i] = lines2[i].replace(p[1], attenuation)
        if 'SAVE_SNAPSHOTS' in lines2[i]:
            p = lines2[i].split()
            lines2[i] = lines2[i].replace(p[1], save_snapshots)
            
with open(f"{SOLVER}/inparam_basic", 'w') as fout:
    for item in lines2:
        fout.write("%s\n" % item)

if sim_type == 'moment':
    with open (f"{SOLVER}/CMTSOLUTION", 'r') as fin:
        file4 = fin.read()
        lines4 = file4.splitlines()
        for i in range(0, len(lines4)):
            if 'latitude' in lines4[i]:
                print(lines4[i])
                p = lines4[i].split(":")
                lines4[i] = lines4[i].replace(p[1].strip(), ev_lat)
                print(lines4[i])
            if 'longitude' in lines4[i]:
                p = lines4[i].split(":")
                lines4[i] = lines4[i].replace(p[1].strip(), ev_lon)
            if 'depth' in lines4[i]:
                p = lines4[i].split(":")
                lines4[i] = lines4[i].replace(p[1].strip(), source_dep)
            if 'Mrr' in lines4[i]:
                p = lines4[i].split(":")
                lines4[i] = lines4[i].replace(p[1].strip(), Mrr)
            if 'Mtt' in lines4[i]:
                p = lines4[i].split(":")
                lines4[i] = lines4[i].replace(p[1].strip(), Mtt)
            if 'Mpp' in lines4[i]:
                p = lines4[i].split(":")
                lines4[i] = lines4[i].replace(p[1].strip(), Mpp)
            if 'Mrt' in lines4[i]:
                p = lines4[i].split(":")
                lines4[i] = lines4[i].replace(p[1].strip(), Mrt)
            if 'Mrp' in lines4[i]:
                p = lines4[i].split(":")
                lines4[i] = lines4[i].replace(p[1].strip(), Mrp)
            if 'Mtp' in lines4[i]:
                p = lines4[i].split(":")
                lines4[i] = lines4[i].replace(p[1].strip(), Mtp)
                
    with open(f"{SOLVER}/CMTSOLUTION", 'w') as fout:
        for item in lines4:
            fout.write("%s\n" % item)
        
with open(f"{SOLVER}/inparam_source", 'r') as fin:
    file3 = fin.read()
    lines3 = file3.splitlines()
    for i in range(0, len(lines3)):
        if 'SOURCE_TYPE' in lines3[i]:
            p = lines3[i].split()
            lines3[i] = lines3[i].replace(p[1], source_type)
        if 'SOURCE_DEPTH' in lines3[i]:
            p = lines3[i].split()
            lines3[i] = lines3[i].replace(p[1], source_dep)
        if 'SOURCE_LAT' in lines3[i]:
            p = lines3[i].split()
            lines3[i] = lines3[i].replace(p[1], ev_lat)
        if 'SOURCE_LON' in lines3[i]:
            p = lines3[i].split()
            lines3[i] = lines3[i].replace(p[1], ev_lon)
        if 'SOURCE_AMPLITUDE' in lines3[i]:
            p = lines3[i].split()
            lines3[i] = lines3[i].replace(p[1], source_amp)

with open(f"{SOLVER}/inparam_source", 'w') as fout:
    for item in lines3:
        fout.write("%s\n" % item)

with open(f"{SOLVER}/inparam_advanced", 'r') as fin:
    file5 = fin.read()
    lines5 = file5.splitlines()
    for i in range(0,len(lines5)):
        if 'SOURCE_FUNCTION' in lines5[i]:
            p = lines5[i].split()
            lines5[i] = lines5[i].replace(p[1], source_function)
        if 'SNAPSHOT_DT' in lines5[i]:
            p = lines5[i].split()
            lines5[i] = lines5[i].replace(p[1], snapshot_dt)
        if 'SNAPSHOTS_FORMAT' in lines5[i]:
            p = lines5[i].split()
            lines5[i] = lines5[i].replace(p[1], snapshots_format)
        if 'F_MIN' in lines5[i]:
            p = lines5[i].split()
            lines5[i] = lines5[i].replace(p[1], f_min)
        if 'F_MAX' in lines5[i]:
            p = lines5[i].split()
            lines5[i] = lines5[i].replace(p[1], f_max)
        if 'F_REFERENCE' in lines5[i] and 'QPL' not in lines5[i]:
            p = lines5[i].split()
            lines5[i] = lines5[i].replace(p[1], f_ref)
            
with open(f"{SOLVER}/inparam_advanced", 'w') as fout:
    for item in lines5:
        fout.write("%s\n" % item)

The next two cells run the AxiSEM mesher by changing to the right directory, calling the submit.csh script, and then tidying up the output

In [58]:
%%bash -s "$MESHER"
echo "This bash script knows about $1"
cd $1
./submit.csh
echo "Mesher complete"

This bash script knows about /Users/katdapre/Documents/Synthetics/axisem-9f0be2f/MESHER
make: Nothing to be done for `all'.
Using external mesh file  github_ice_shell.bm
[1] 18009
xmesh submitted, output in "OUTPUT"
After the run, move the mesh to a new directory <meshdir> via:
./movemesh <meshdir>
This will be located in ../SOLVER/MESHES/<meshdir>
Yeah that's done
     Meshing started on 05/20/2022 at 15h 17min

     CLOCKS module Clocks.F90,v 3.0 2014/10/19
     Realtime clock resolution=   1.0000000474974513E-003 (        1000  ticks/sec)
Reading inparam_mesh... done
 
 PREDEFINED MODEL/SIMULATION PARAMETERS
 Background model                 : external
 Dominant period [s]              :   0.50000000000000000     
 Elements per dominant wavelength :    1.5000000000000000     
 Courant number                   :   0.60000002384185791     
 coarsening levels                :            0
 processors used in solver        :            1
 outer radius [m]                 :    252000.000

rm: No match.
rm: No match.
rm: No match.


In [59]:
%%bash -s "$MESHER" "$run_name"
cd $1
./movemesh.csh $2

Moving mesh from . to ../SOLVER/MESHES/github_2Hz_test13
MESHER finished smoothly...
Moving mesh to /Users/katdapre/Documents/Synthetics/axisem-9f0be2f/SOLVER/MESHES/github_2Hz_test13
external
Copying external model file github_ice_shell.bm
Contents in /Users/katdapre/Documents/Synthetics/axisem-9f0be2f/SOLVER/MESHES/github_2Hz_test13 :
1dmodel_axisem.bm
1dmodel_yspec.bm
Code
OUTPUT
external_model.bm
inparam_mesh
mesh_courant.vtk
mesh_domaindecomposition.vtk
mesh_domaindecomposition_cell.vtk
mesh_dt.vtk
mesh_eltype.vtk
mesh_hmax.vtk
mesh_hmin.vtk
mesh_params.h
mesh_period.vtk
mesh_pts_wavelength.vtk
mesh_rho.vtk
mesh_vp.vtk
mesh_vs.vtk
meshdb.dat0000
radial_velocity.dat
Total size: 453M	. 
DONE.


The next cell switches to the solver directory and runs the submit.csh script there

In [60]:
%%bash -s "$SOLVER" "$run_name"
cd $1
touch main.f90
make all
cd $1
./submit.csh $2

gfortran -O3 -march=native  -Dserial -Dsolver -c  main.f90
gfortran -O3  -pthread -o axisem background_models.o commpi.o commun.o get_model.o lateral_heterogeneities.o meshes_io.o nc_helpers.o nc_routines.o nc_snapshots.o parameters.o time_evol_wave.o analytic_mapping.o analytic_semi_mapping.o analytic_spheroid_mapping.o apply_masks.o attenuation.o clocks.o clocks_wrapper_solver.o data_comm.o data_heterogeneous.o data_io.o data_matr.o data_mesh.o data_pointwise.o data_proc.o data_source.o data_spec.o data_time.o def_grid.o def_precomp_terms.o get_mesh.o global_parameters.o interpolation.o kdtree2.o list.o main.o pointwise_derivatives.o rotations.o seismograms.o source.o stiffness_di.o stiffness_fluid.o stiffness_mono.o stiffness_quad.o subpar_mapping.o unrolled_loops.o utlity.o wavefields_io.o ftz.o pthread.o 
Using mesh  MESHES/github_2Hz_test13
copying mesh_params.h from  MESHES/github_2Hz_test13
make: Nothing to be done for `all'.
Receiver file type: colatlon
Source file: inparam_so

The next cell runs post-processing on solver output

In [69]:
%%bash -s "$SOLVER" "$run_name"
cd $1/UTILS
make all
cd $1/$2
./post_processing.csh

make: Nothing to be done for `all'.


>>>>>>>>> AXISEM POST PROCESSING <<<<<<<<<


All post-processed data in: ./Data_Postprocessing
Homedir: /Users/katdapre/Documents/Synthetics/axisem-9f0be2f/SOLVER/github_2Hz_test13 simdir1: ./

%%%%%%%%% PROCESSING seismograms/wavefields %%%%%%%%%
.... output in ./Data_Postprocessing/OUTPUT_postprocessing ....
xpost_processing exited with error
check output for details:
./Data_Postprocessing/OUTPUT_postprocessing


At line 349 of file post_processing.F90 (unit = 60, file = './/Data/recfile_0001_disp.dat')
Fortran runtime error: End of file

Error termination. Backtrace:
#0  0x10d8ca4dc
#1  0x10d8cb395
#2  0x10d8cbf55
#3  0x10de44de3
#4  0x10de3b73e
#5  0x10de3c9b0
#6  0x10de430c2
#7  0x10d193de7
#8  0x10d197aee
